In [ ]:
import pandas as pd


In [ ]:
# Read all three text files into DataFrames
#all_df = pd.read_csv(
   # "All_all.use.final.txt",
   # sep="\t",
   # quotechar='"',
   # engine='python',
   # on_bad_lines='skip'  # replaces error_bad_lines
#)
import sys
import csv
csv.field_size_limit(sys.maxsize)
#dem_df = pd.read_csv(
   # "Dem_all.use.final.txt",
   # sep="\t",
   # quotechar='"'
#)

#dia_df = pd.read_csv(
    #"Dia_all.use.final.txt",
   # sep="\t",
   # quotechar='"'
#)

dia_df = pd.read_csv(
    'Dia_all.use.final.txt',
    sep='\t',
    quotechar='"'
)

In [ ]:
import warnings

with warnings.catch_warnings(record=True) as w:
    dia_df = pd.read_csv(
        "Dia_all.use.final.txt",
        sep="\t",
        quotechar='"',
        engine='python',
        on_bad_lines='skip'
    )
    print(f"{len(w)} warnings captured.")

In [ ]:
dem_df.info()

In [ ]:
dem_df.head()

In [ ]:
dem_df_c=dem_df.copy()

In [ ]:
dem_df_c.head()

In [ ]:
dem_df_c["FirstNAFL.before.ICD11"].value_counts()

In [ ]:
#FirstNAFL.before.ICD11
dem_df_c= dem_df_c[dem_df_c["FirstNAFL.before.ICD11"]=="y"]

In [ ]:
dem_df_c.shape

In [ ]:
dem_df_c.columns

In [ ]:
#Check NA percentage
na_percent = dem_df_c.isna().mean().sort_values(ascending=False) * 100
print("NA Percentage by Column:")
print(na_percent)

In [ ]:
#select columns
selected_columns = ['StudyID', 'FirstNAFL.Age.90', 'Gender_Legal_Sex']  # Replace 'BMI_Category' with actual column name if different
dem_df_c= dem_df_c[selected_columns]

# Display the filtered DataFrame
dem_df_c.head()

In [ ]:
#check gender
dem_df_c["Gender_Legal_Sex"].value_counts()

In [ ]:
#check age
age_bins = pd.cut(dem_df_c['FirstNAFL.Age.90'], bins=range(0, 101, 10), right=False)
age_counts = age_bins.value_counts().sort_index()

# Display the result
print("Age distribution in 10-year intervals:")
print(age_counts)

In [ ]:
dem_df_c=dem_df_c[dem_df_c["FirstNAFL.Age.90"]>=30]

In [ ]:
dem_df_c

In [ ]:
# Check NA rate (percentage of missing values per column)
na_rate = dem_df_c.isna().mean() * 100
print("NA Rate (%):")
print(na_rate)


In [ ]:
dem_df_c[dem_df_c.duplicated()].head()
#no duplicates

In [ ]:
#not necessary
# Handle high-NA columns (optional threshold: 90%)
#high_na_cols = na_percent[na_percent > 90].index
#dem_df = dem_df.drop(columns= high_na_cols)
#print(f"\n Dropped columns with >90% NA: {list(high_na_cols)}")

In [ ]:
# Drop duplicate rows
dem_df_c.shape


In [ ]:
dem_df_c_encoded=pd.get_dummies(dem_df_c, columns=["Gender_Legal_Sex"], drop_first=True)

In [ ]:
dem_df_c_encoded

In [ ]:
dem_df_c_encoded.to_csv("dem_df_c_encoded.csv", index=False)

In [ ]:
dem_df_c.to_csv("dem_df_clean.csv", index=False)

In [ ]:
dia_df.shape

In [ ]:
dia_df_c=dia_df.copy()

In [ ]:
dia_df_c.head()

In [ ]:
rows, cols = dia_df.shape
print(f"The dia_df has {rows} rows and {cols} columns.")

In [ ]:
dia_df_c.columns

In [ ]:
na_percent = dia_df_c.isna().mean().sort_values(ascending=False) * 100
print("NA Percentage by Column:")
print(na_percent)

In [ ]:
#check/drop duplications, keep the first occurrence 

dia_df_c[dia_df_c.duplicated()].head()

In [ ]:
duplicates = dia_df_c[dia_df_c.duplicated(keep=False)]
print(f"Total duplicated rows: {len(duplicates)}")

shown = 0
for _, group in duplicates.groupby(list(dia_df_c.columns)):
    if len(group) > 1:
        display(group)
        shown += 1
        if shown == 5:
            break


In [ ]:
dia_df_c = dia_df_c.drop_duplicates(keep='first')
print(f"Deduplicated shape: {dia_df_c.shape}")

In [ ]:
dia_df_c["Code_Type"].value_counts()

In [ ]:
dia_df_c.dtypes

In [ ]:
#keep only ICD10
dia_df_c=dia_df_c[(dia_df_c["Code_Type"]== "ICD10") & (dia_df_c["Dia.before.ICD11"]== "y")]
dia_df_c["Code_Type"].value_counts()

In [ ]:
dia_df_c["Dia.before.ICD11"].value_counts()

In [ ]:
#drop dia.before.ICD11=n
dia_df_c=dia_df_c.drop(columns="Dia.before.ICD11")
dia_df_c

In [ ]:
dia_df[dia_df["Code"]==""]

In [ ]:
#exclude patients with worse conditions before the diagnose of NAFL
worse_conditions = [
    'K75.81', 'K75.89',
    'K72.00', 'K72.01', 'K72.10', 'K72.11', 'K72.90', 'K72.91',
    'K74.0', 'K74.00', 'K74.01', 'K74.02', 'K74.1', 'K74.2', 'K74.60', 'K74.69',
    'C22.0'
]

pre_nafl_df = dia_df_c[dia_df_c["Dia.daysfrom_firstNAFL"] < 0]

for code in worse_conditions:
    condition = pre_nafl_df["Code"].astype(str).str.startswith(code)
    matched_df = pre_nafl_df[condition]
    patient_count = matched_df["StudyID"].nunique()
    print(f"{code}: {patient_count} patients")

In [ ]:
worse_conditions = [
    'K75.81', 'K75.89',
    'K72.00', 'K72.01', 'K72.10', 'K72.11', 'K72.90', 'K72.91',
    'K74.0', 'K74.00', 'K74.01', 'K74.02', 'K74.1', 'K74.2', 'K74.60', 'K74.69',
    'C22.0','C22.1', 'C22.2', 'C22.3', 'C22.4', 'C22.7', 'C22.8','C22.9'	
]



# Step 1: Filter diagnoses before NAFL diagnosis
pre_nafl_df = dia_df_c[dia_df_c["Dia.daysfrom_firstNAFL"] < 0]
clean_df = dia_df_c[~dia_df_c["StudyID"].isin(exclude_ids)]
# Step 2: Initialize empty set for patient IDs to exclude
exclude_ids = set()

print("Number of patients with each condition before NAFL diagnosis:\n")
for code in worse_conditions:
    matched = pre_nafl_df[pre_nafl_df["Code"].astype(str) == code]
    patient_ids = matched["StudyID"].unique()
    print(f"{code}: {len(patient_ids)} patients")
    exclude_ids.update(patient_ids)

# Step 3: Drop these patients from the original DataFrame
dia_df_c = dia_df_c[~dia_df_c["StudyID"].isin(exclude_ids)]

In [ ]:
#checking age groups
age_bins = pd.cut(dia_df_c['Dia.Age.90'], bins=range(0, 101, 10), right=False)
age_counts = age_bins.value_counts().sort_index()

# Display the result
print("Age distribution in 10-year intervals:")
print(age_counts)

In [ ]:
#keep patients who have diagnoses history for three years before the diagnosis of NAFL
dia_df_c = dia_df_c[
    (dia_df_c["Dia.daysfrom_firstNAFL"] < 0) & 
    (dia_df_c["Dia.daysfrom_firstNAFL"] >= -1095)
]

In [ ]:
dia_df_c.shape

In [ ]:
dia_df_c.to_csv("dia_df_clean.csv", index=False)

In [ ]:
encoded_dia_df = pd.get_dummies(
    dia_df_c,
    columns=['Diagnosis_Name', 'Code_Type', 'Diagnosis_Flag', 'Inpatient_Outpatient'],
    drop_first=True  # set to True if you want to avoid multicollinearity
)

# Display the transformed DataFrame
encoded_dia_df.head()

In [ ]:
encoded_dia_df.to_csv("dia_df_clean_encode.csv", index=False)